In [141]:
import re
import numpy as np
import math

Rafael Hinojosa López

A01705777

28-05-2023

Desarrollo de aplicaciones avanzadas de ciencias computacionales (Gpo 301)

##### Read texts

In [142]:
with open("./text_3.txt", 'r', encoding='utf-8') as f:
    text1 = f.read()

In [143]:
with open("./text_4.txt", 'r', encoding='utf-8') as f:
    text2 = f.read()

##### Preprocess text

In [144]:
text1 = text1.lower()
text2 = text2.lower()

text1 = re.findall(r"[A-Za-z=+-/!*]+|\w+", text1)
text2 = re.findall(r"[A-Za-z=+-/!*]+|\w+", text2)

combined_text = text1 + text2
combined_text

['a', 'c', 'b', 'a', 'd', 'c', 'b', 'a', 'c', 'a', 'b', 'd', 'b', 'a', 'b']

###### Create one directed graph per text where vertex (key of the dict) _v_ represents a word in the text and contains a list of all the words that appear right after that word

In [145]:
words1 = {}
words2 = {}

for word in combined_text:
    words1[word] = []
    words2[word] = []

# Create graph for text1
for i in range(1, len(text1)):
    origin_word = text1[i-1]
    destination_word = text1[i]
    words1[origin_word].append(destination_word)

# Create graph for text2
for i in range(1, len(text2)):
    origin_word = text2[i-1]
    destination_word = text2[i]
    words2[origin_word].append(destination_word)
    
print(words1)
print(words2)

{'a': ['c', 'd'], 'c': ['b'], 'b': ['a'], 'd': ['c']}
{'a': ['c', 'b', 'b'], 'c': ['a'], 'b': ['a', 'd', 'a'], 'd': ['b']}


###### Crear un arreglo donde se mapeen las palabras con su posicion en el grafo

In [146]:
words1_index = {}
words2_index = {}

index = 0
for word in words1:
    words1_index[word] = index
    index += 1

index = 0
for word in words2:
    words2_index[word] = index
    index += 1

print(words1_index)
print(words2_index)

{'a': 0, 'c': 1, 'b': 2, 'd': 3}
{'a': 0, 'c': 1, 'b': 2, 'd': 3}


###### Create transition matrices

- inicializar un array de length palabras con 0s
- llenar las que ya se tienen guardadas en su indice correspondiente y ya!! hacer append de la lista a la matriz wuuu
/ ok necesitamos que las matrices y asi... estar tratando con todas las palabras, no solo con las actuales...

con esto ponemos ok... mat1[map[i]] = 1 / total o asi... o sea aja... 




In [147]:
mat1 = []
mat2 = []

len1 = len(words1)
len2 = len(words2)

for word in words1:
    list1 = [0 for i in range(len1)]

    val1 = 0
    if len(words1[word]) != 0:
        val1 = 1 / len(words1[word])

    for next_word in words1[word]:
        next_word_index = words1_index[next_word]
        list1[next_word_index] += val1        

    mat1.append(list1)


for word in words2:
    list2 = [0 for i in range(len2)]
    
    val2 = 0
    if len(words2[word]) != 0:
        val2 = 1 / len(words2[word])

    for next_word in words2[word]:
        next_word_index = words2_index[next_word]
        list2[next_word_index] += val2        

    mat2.append(list2)

mat1 = np.array(mat1)
mat2 = np.array(mat2)

print(mat1)
print(mat2)

[[0.  0.5 0.  0.5]
 [0.  0.  1.  0. ]
 [1.  0.  0.  0. ]
 [0.  1.  0.  0. ]]
[[0.         0.33333333 0.66666667 0.        ]
 [1.         0.         0.         0.        ]
 [0.66666667 0.         0.         0.33333333]
 [0.         0.         1.         0.        ]]


##### Calculate similarity

In [148]:
BT = mat2.transpose()
C = np.matmul(BT, mat1)
prod_int = np.trace(C)

norm_A = math.sqrt(np.trace(np.matmul(mat1.transpose(), mat1)))
norm_B = math.sqrt(np.trace(np.matmul(mat2.transpose(), mat2)))

cos_ang = prod_int / (norm_A * norm_B)


print('A | Mat1 = \n' + str(mat1) + '\n')
print('B | Mat2 = \n' + str(mat2) + '\n')
print('BT | Transpose of Mat2 = \n' + str(BT) + '\n')
print('C = ' + str(C) + '\n')
print('Prod int = ' + str(round(prod_int, 4)) + '\n')
print('Norm A | Norm Mat1 = ' + str(round(norm_A, 4)) + '\n')
print('Norm B | Norm Mat2 = ' + str(round(norm_B, 4)) + '\n')
print('Cosine angle = ' + str(round(cos_ang, 4)) + '\n')
print('The similarity of both texts is ' + str(round(cos_ang, 4) * 100) + '%' + ', based on the cosine distance of transition matrices.')

A | Mat1 = 
[[0.  0.5 0.  0.5]
 [0.  0.  1.  0. ]
 [1.  0.  0.  0. ]
 [0.  1.  0.  0. ]]

B | Mat2 = 
[[0.         0.33333333 0.66666667 0.        ]
 [1.         0.         0.         0.        ]
 [0.66666667 0.         0.         0.33333333]
 [0.         0.         1.         0.        ]]

BT | Transpose of Mat2 = 
[[0.         1.         0.66666667 0.        ]
 [0.33333333 0.         0.         0.        ]
 [0.66666667 0.         0.         1.        ]
 [0.         0.         0.33333333 0.        ]]

C = [[0.66666667 0.         1.         0.        ]
 [0.         0.16666667 0.         0.16666667]
 [0.         1.33333333 0.         0.33333333]
 [0.33333333 0.         0.         0.        ]]

Prod int = 0.8333

Norm A | Norm Mat1 = 1.8708

Norm B | Norm Mat2 = 1.7638

Cosine angle = 0.2525

The similarity of both texts is 25.25%, based on the cosine distance of transition matrices.


##### Reflexiones

1. Adecua tu programa para encontrar similitud entre dos códigos hechos en Python. ¿Qué preprocesamiento tendrías que hacer con los códigos antes de compararlos? (Por ejemplo, a = b + c y r = m + n se tomarían como completamente disintos. ¿Cómo podrías evitar eso?

El programa fue adecuado para identificar símbolos que el lenguaje Python utiliza como: '+', '-', '/', '*', '=', '!'. A pesar de que parece que la solución será más precisa, no es así, ya que al no contar con un _lexer_ para tokenizar correctamente los símbolos y las palabras reservadas o variables que se tienen en el código, algunas palabras se mezclaron con los símbolos, lo cual hizo que la precisión con la que se calcula la distancia disminuya hasta en un 100% (en el extremo peor de los casos).

Para evitar que _a = b + c_ y _r = m + n_ sean diferentes, es necesario tokenizar ambas expresiones para que, traducido sea: _variable = variable + variable_ en ambos casos. 

Esto implicaría implementar o utilizar un _lexer_ y _mapear_ las palabras con el tipo de expresiones que son en Python. Por ejemplo, en vez de guardar el nombre de una variable, guardar el nombre _variable_, con el fin de enfocarse en que la palabra en cuestión es una variable sin importar el nombre que le asignaron. 

Al tokenizar, podremos obtener la similitud que hay entre dos códigos con respecto al orden en el que los tokens aparecen en el código, suponiendo que utilizamos esta técnica de matrices de transición. 


2. Realiza distintas pruebas con distintos códigos hechos en Python. ¿Crees que esta técnica es adecuada para encontrar la similitud entre códigos?¿Es más eficiente que la técnica de similitud por distribuciones de probabilidad?

Tomando como ejemplo los archivos sol_1.txt y sol_2.txt, la similitud que obtenemos con ambas soluciones es la siguiente:
- Similitud por matrices de transición = 20.6%
- Similitud por distribuciones de probabilidad = 61.4%

Se observa que la similitud calculada por distribuciones de probabilidad es ~3 veces más alta que aquella calculada por matrices de transición en este caso. Ambos métodos calculan la similitud de maneras diferentes: uno basado en qué tan parecidos son los textos con respecto a las palabras que contienen y su número de repeticiones (distribuciones de probabilidad); y el otro, basado en el orden de las palabras (matrices de transición). 

Una desventaja de este método en particular es que si el orden del código cambia, la similitud también puede cambiar o no ser completamente precisa (comparar sol_5.txt y sol_6.txt).

Si se tokenizara el código correctamente, considero que ambos métodos pueden ser útiles para encontrar la similitud de dos códigos de Python. Si se quiere medir la similitud léxica, la distribución de probabilidad es la mejor opción; en cambio, si se quiere medir la similitud en el orden del código, la mejor opción es utilizar matrices de transición.
